# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-10-25&until-occurred-date=2021-11-01


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

15801 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-25&until-occurred-date=2021-11-01
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=7ab73589-f8d5-4bb6-b20f-5f76d850ecbb&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-25&until-occurred-date=2021-11-01
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=5a38f11e-44dd-4189-a176-02263ffad9d4&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-25&until-occurred-date=2021-11-01
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-10-18.csv
failed to load preprint_tweets_2021-10-25.csv
failed to load preprint_tweets_2021-07-29.csv
failed to load preprint_tweets_2021-09-06.csv
failed to load preprint_tweets_2021-09-13.csv
failed to load .DS_Store
failed to load preprint_tweets_2021-10-23.csv
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-10-05.csv
failed to load preprint_tweets_2021-10-11.csv
failed to load preprint_tweets_2021-10-04.csv
failed to load preprint_tweets_2021-10-06.csv
failed to load preprint_tweets_2021-10-07.csv
failed to load preprint_tweets_2021-07-26.csv
failed to load preprint_tweets_2021-09-20.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.08.24.21262415...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264262...
REST API query complete  200
REST API query started for 10.1101/2021.09.16.21263684...
REST API query complete  200
REST API query started for 10.1101/2021.09.13.21262182...
REST API query complete  200
REST API query started for 10.1101/2021.09.22.21263977...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264260...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.02.16.21251535...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.09.30.462488...
REST API query complete  200
REST API query started for 10.1101/2021.08.30.21262866...
REST API query complete  200
REST API query started for 10.1101/2021.08.19

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.10.26.465776,202,bioRxiv,Molecular Biology,False,Artificial intelligence reveals nuclear pore c...,"[{'name': 'Shyamal Mosalaganti'}, {'name': 'Ag...",<p>Nuclear pore complexes (NPCs) mediate nucle...,2021-10-27
1,10.1101/2021.10.24.465614,100,bioRxiv,Developmental Biology,False,Tension-driven multi-scale self-organisation i...,"[{'name': 'Qiyan Mao'}, {'name': 'Achyuth Acha...",<p>Human muscle is a hierarchically organised ...,2021-10-25
2,10.1101/2021.10.21.21265342,78,medRxiv,Genetic and Genomic Medicine,False,Linking common and rare disease genetics throu...,"[{'name': 'Olivier B. Bakker'}, {'name': 'Anni...",<p>Genetic variants identified through genome-...,2021-10-26
3,10.1101/2021.10.29.466401,58,bioRxiv,Microbiology,True,SARS-CoV-2 spike protein as a bacterial lipopo...,"[{'name': 'Firdaus Samsudin'}, {'name': 'Palur...",<p>Accumulating evidence indicates a potential...,2021-10-29
4,10.1101/2021.10.26.21265525,58,medRxiv,Epidemiology,True,COVCOG 1: Factors predicting Cognitive Symptom...,"[{'name': 'Panyuan Guo'}, {'name': 'Alvaro Ben...","<p>Since its first emergence in December 2019,...",2021-10-26
5,10.1101/2021.10.27.466055,54,bioRxiv,Bioinformatics,True,Phylogenetic analysis and in silico studies li...,"[{'name': 'Anna Bertelli'}, {'name': ""Pasquali...",<p>Genotype screening was implemented in Italy...,2021-10-29
6,10.1101/2021.10.29.21265511,53,medRxiv,Oncology,True,Blood cytokine analysis suggests that SARS-CoV...,"[{'name': 'Fien HR De Winter'}, {'name': 'An H...","<p>Cytokines, chemokines and (angiogenic) grow...",2021-10-29
7,10.1101/2021.10.25.465200,52,bioRxiv,Immunology,False,Cell-extrinsic autophagy in mature adipocytes ...,"[{'name': 'Felix Clemens Richter'}, {'name': '...",<p>Autophagy is a critical cellular recycling ...,2021-10-26
8,10.1101/2021.10.27.466091,43,bioRxiv,Biochemistry,False,Cryo-electron microscopy structure of the H3-H...,"[{'name': 'Kayo Nozawa'}, {'name': 'Yoshimasa ...","<p>The canonical nucleosome, which represents ...",2021-10-28
9,10.1101/2021.10.29.466387,43,bioRxiv,Biochemistry,False,Discovery of a conserved rule behind the assem...,"[{'name': 'Edward M Germany'}, {'name': 'Yue D...","<p>Gram-negative bacteria, mitochondria and ch...",2021-10-29


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

182 preprints (including 38 covering SARS-CoV-2, 146 from bioRxiv and 36 from medRxiv) published in the last 7 days before 2021-11-01 had been tweeted at least 3 times (maximum 202).